# 提高准确率

之前的模型的准确率似乎有一些太低了,大约只有28%-30%左右,这样可能会在分析中产生一些新的问题.而我们期待的正确率可以在50%附近,通过提高模型的复杂度自然可以轻易的做到这一点,但是那样在计算上会花费太多时间,为了节约算力,我考虑使用更好的optimizer和scheduler,也就是利用超参数实现准确率的提高.

## MLP网络---不变
这里依旧采用之前的MLP模型

In [14]:
import torch.nn as nn
import torch.nn.functional as F

''' MLP '''
class MLP(nn.Module):
    def __init__(self, channel=3, num_classes=10, im_size=(32, 32)):
        super(MLP, self).__init__()
        self.fc_1 = nn.Linear(im_size[0] * im_size[1]*channel, 512)
        self.fc_2 = nn.Linear(512, 512)
        self.fc_3 = nn.Linear(512, num_classes)

    def forward(self, x):
        out = x.view(x.size(0), -1)
        out = F.relu(self.fc_1(out))
        out = F.relu(self.fc_2(out))
        out = self.fc_3(out)
        return out

## 新的模型

采用**Adam**作为optimizer,采用**torch.optim.lr_scheduler.LambdaLR**实现线性动态的对学习率的调整

In [15]:
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import torchvision
from torchvision.datasets import CIFAR10
import numpy as np
import random
from get_rank_cpu_0904 import Effective_Ranks, get_Effective_Ranks
from torch.optim.lr_scheduler import LambdaLR   # 这个用来实现学习率的线性变化
from torch.optim.lr_scheduler import CosineAnnealingLR  # 实现cos函数式的变化
from load_dataset import get_data_loader

def get_scale(scale, writer,  
            device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"), train_loader=None, valid_loader=None, test_loader=None, channel=3,
            num_classes=10,
            im_size=(32, 32),
            # 可能需要的超参数
            momentum = 0.9,
            weight_decay = 0.0005,
            initial_lr = 0.1,
            num_epochs = 100):

  # -------------------------------------------------创建模型实例并将其移至 GPU -----------------------------------------------------
  # model = MLP(channel=channel, num_classes=num_classes, im_size=im_size)
  model = MLP()
  model.to(device)

  # 定义损失函数/优化器/学习率
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(
    model.parameters(),
    lr=initial_lr,  # 这里我只想关心学习率的影响,所以其他的参数先采用默认值
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0.0005)
  # scheduler = LambdaLR(optimizer, lr_lambda=lambda epoch: 1/(epoch+1))
  scheduler = CosineAnnealingLR(optimizer, T_max=20)

  # 记录训练过程中的准确率和损失
  tag_loss = "train_loss_{}".format(scale)
  tag_acc = "train_accuracy_{}".format(scale)

  # 训练循环...
  final_train_loss = 0
  final_train_acc = 0
  best_train_acc = 0.0  # 用于跟踪一个scale也就是一个完整的train的过程中的最佳准确率

  # 验证步骤...
  # 记录验证的次数
  valid_correct = 0
  valid_total = 0
  best_valid_acc = 0.0
  val_loss = "val_loss_{}".format(scale)
  val_acc = "val_accuracy_{}".format(scale)
  acc = 0.0
  loss = 1.0

  for epoch in range(num_epochs):

    # ----------------------------首先是训练过程----------------------------
    train_correct = 0
    train_total = 0
    for batch_idx, (imgs, targets) in enumerate(train_loader):
      imgs, targets = imgs.to(device), targets.to(device)
      outputs = model(imgs)
      loss = criterion(outputs, targets)
      _, predicted = torch.max(outputs, 1)
      train_total += targets.size(0)
      train_correct += (predicted == targets).sum().item()
      optimizer.zero_grad() # W_new = W_old + \gamma \times \deta W_old,这里相当于先要求\gamma 为零矩阵
      loss.backward()   # 使用backward得到了\deta W_old
      optimizer.step()  # 选择合适的\gamma 得到了W_new
      scheduler.step()  # \gamma 事实上是一个依赖于learning_rate的一个矩阵函数,这里更新了learning_rate,而且learning_rate的更新是依赖于epoch的

    acc = train_correct / train_total
    writer.add_scalar(tag_loss, loss.item(), epoch)
    writer.add_scalar(tag_acc, acc, epoch)
    # 更新模型权重并记录到Tensorboard
    for name, param in model.named_parameters():
      writer.add_histogram(name, param.clone().cpu().data.numpy(), epoch)
    final_train_acc = train_correct / train_total
    # 更新最佳准确率,以及对应的loss
    if final_train_acc >= best_train_acc:
      best_train_acc = final_train_acc
      best_train_loss = loss.item()
    final_train_loss = loss.item()
    print("Epoch [{}/{}], Final Train Loss: {:.4f}, Final Train Accuracy: {:.2f}%, Best Train Accuracy: {:.2f}, Loss to best acc:{:.2f}%".format(epoch+1, num_epochs, final_train_loss, final_train_acc * 100, best_train_acc * 100, best_train_loss),"\n")
    # 记录学习率(learning_rate)的更新情况
    writer.add_scalar("learning_rate",optimizer.param_groups[0]['lr'])

    # -------------------------然后进入了验证步骤---------------------------------
    for batch_idx, (imgs, targets) in enumerate(valid_loader):
      imgs, targets = imgs.to(device), targets.to(device)
      outputs = model(imgs)
      loss = criterion(outputs, targets)
      _, predicted = torch.max(outputs, 1)
      valid_total += targets.size(0)
      valid_correct += (predicted == targets).sum().item()
      acc = valid_correct / valid_total

    writer.add_scalar(val_loss, loss.item(), epoch)
    writer.add_scalar(val_acc, acc, epoch)
    # 记录valid中的最终情况
    final_valid_acc = acc
    final_valid_loss = loss
    # 更新最佳准确率,以及对应的loss
    if final_valid_acc >= best_valid_acc:
      best_valid_acc = final_valid_acc
      best_valid_loss = loss.item()
    final_valid_loss = loss.item()
    print("Epoch [{}/{}], Final Valid Loss: {:.4f}, Final Valid Accuracy: {:.2f}%, Best Valid Accuracy: {:.2f}%, Loss to best Valid accuracy:{:.2f}".format(epoch+1, num_epochs, final_valid_loss, final_valid_acc * 100, best_valid_acc * 100, best_valid_loss),"\n")


  # 记录train过程中的最终情况
  writer.add_scalar("final_train_loss", final_train_loss, scale)
  writer.add_scalar("final_train_acc", final_train_acc, scale)
  writer.add_scalar("best_train_acc", best_train_acc, scale)
  writer.add_scalar("best_train_loss", best_train_loss, scale)
  # 记录valid的最终情况
  writer.add_scalar("final_valid_loss", final_valid_loss, scale)
  writer.add_scalar("final_valid_acc", final_valid_acc, scale)
  writer.add_scalar("best_valid_acc", best_valid_acc, scale)
  writer.add_scalar("best_valid_loss", best_valid_loss, scale)

  # 保存模型参数
  filename = "LP-RL-model_scale={}.pth".format(scale)
  torch.save(model.state_dict(), filename)
  print("模型参数已保存为{}".format(filename), '\n')

  # 完成各种工作，记录结果...
  normal_dict = {
      "训练的模型的保留路径": filename,
      "在测试集中的最终损失": final_valid_loss,
      "在测试集中的最终损失":best_valid_loss,
      "在测试集中的最终准确率": final_valid_acc,
      "在测试集中的最佳准确率": best_valid_acc,
      "在训练集中的最终准确率": final_train_acc,
      "在训练集中的最佳准确率": best_train_acc,
      "在训练集中的最终损失": final_train_loss,
      "在训练集中的最佳损失":best_train_loss
  }
  return normal_dict


In [16]:
def get_rank(scale, writer, device, train_loader):
  get_cifar10_gpu = get_Effective_Ranks(train_dataloader=train_loader)
  rk, Rk = get_cifar10_gpu.train_rk, get_cifar10_gpu.train_Rk
  rk_max_value = max(rk)  # 找到列表中的最大值
  rk_max_index = rk.index(rk_max_value)  # 找到最大值对应的索引
  Rk_value_max_rk_index = Rk[rk_max_index]
  # Rk_max_value = max(Rk)  # 找到列表中的最大值
  # Rk_max_index = Rk.index(Rk_max_value)  # 找到最大值对应的索引
  # print(rk_max_index, rk_max_value, '\n', Rk_max_index, Rk_max_value)

  # 记录的标记
  # 构建动态字符串
  index_rk = "rk_max_index_scale"
  # index_Rk = "Rk_max_index_sacle={}".format(scale)
  value_rk = "rk_max_value_scale"
  value_Rk = "Rk_max_value_sacel"
  value_Rk_index_max_rk = "value_Rk_index_max_rk_sacle" # 这里记录的是rk最大的时候的k对应的Rk的value
  r_0 = "r0_sacle"
  writer.add_scalar(index_rk, rk_max_index, scale)
  # writer.add_scalar(index_Rk, Rk_max_index, scale)
  writer.add_scalar(value_rk, rk_max_value, scale)
  # writer.add_scalar(value_Rk, Rk_max_value, scale)
  writer.add_scalar(r_0, scale)
  writer.add_scalar(value_Rk_index_max_rk, Rk_value_max_rk_index, scale)

  # 建立字典，保存结果
  rank_dict = {index_rk:rk_max_index,
          value_rk:rk_max_value,
          value_Rk:Rk_value_max_rk_index,
          r_0:rk[0]}
  return rank_dict

In [17]:
# ------------------------------------训练开始前的基本设置---------------------------------------------------
# 设置设备和种子
device_cuda = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# 设置随机种子
seed = 10  # 你可以选择任何整数作为种子值

# 设置PyTorch随机种子
torch.manual_seed(seed)

# 设置CUDA随机种子（如果使用GPU）
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# 设置Numpy随机种子
np.random.seed(seed)

# 设置Python随机种子
random.seed(seed)


# ---------------------------------- 设置结束----------------------------------------------------------------------------------

# 定义初始的scale范围
scale_min = 0.1
scale_max = 0.9
scale_list = [round(x * 0.1, 1) for x in range(int(scale_min * 10), int(scale_max * 10) + 1)]
# print(scale_list)
my_list = []
# 添加tensorboard
writer_new = SummaryWriter("logs-MLP-RL")

for scale in scale_list:
  train_loader, valid_loader, test_loader = get_data_loader(scale, valid_scale=0.1)
  print("--------------------开始scale={}的情况---------------------".format(scale), "\n")
  print("首先得到了有效秩的情况 \n")
  dic = {}
  dic['rank_sacle={}'.format(scale)] = get_rank(scale=scale,device=device_cuda, train_loader=train_loader, writer=writer_new)
  print(dic['rank_sacle={}'.format(scale)])
  dic['sacle={}'.format(scale)] = get_scale(scale=scale, writer=writer_new, train_loader=train_loader, valid_loader=valid_loader)
  print(dic['sacle={}'.format(scale)])
  my_list.append(dic)

writer_new.close()

Files already downloaded and verified
Files already downloaded and verified
--------------------开始scale=0.1的情况--------------------- 

首先得到了有效秩的情况 

{'rk_max_index_scale': 1585, 'rk_max_value_scale': 3559136.0927861114, 'Rk_max_value_sacel': 958.9959793243203, 'r0_sacle': 2.0244539429268635}
Epoch [1/100], Final Train Loss: 2.3062, Final Train Accuracy: 10.60%, Best Train Accuracy: 10.60, Loss to best acc:2.31% 

Epoch [1/100], Final Valid Loss: 2.3001, Final Valid Accuracy: 10.60%, Best Valid Accuracy: 10.60%, Loss to best Valid accuracy:2.30 

Epoch [2/100], Final Train Loss: 2.3002, Final Train Accuracy: 9.86%, Best Train Accuracy: 10.60, Loss to best acc:2.31% 

Epoch [2/100], Final Valid Loss: 2.3025, Final Valid Accuracy: 10.35%, Best Valid Accuracy: 10.60%, Loss to best Valid accuracy:2.30 

Epoch [3/100], Final Train Loss: 2.3039, Final Train Accuracy: 9.81%, Best Train Accuracy: 10.60, Loss to best acc:2.31% 

Epoch [3/100], Final Valid Loss: 2.3051, Final Valid Accuracy: 10.27